In [12]:
%pwd

'C:\\'

In [ ]:
"""
Platformer Game
"""
from playsound import playsound
import arcade
import os
import time
import pygame
# Constants (alter for your game or delete)
SCREEN_WIDTH = 1080
SCREEN_HEIGHT = 720
SCREEN_TITLE = "Definitely Minecraft"
CHARACTER_SCALING = 1
TILE_SCALING = 0.5
COIN_SCALING = .5
PLAYER_MOVEMENT_SPEED = 5
GRAVITY = .75
PLAYER_JUMP_SPEED = 16

# How many pixels to keep as a minimum margin between the character
# and the edge of the screen.
LEFT_VIEWPORT_MARGIN = 200
RIGHT_VIEWPORT_MARGIN = 200
BOTTOM_VIEWPORT_MARGIN = 75
TOP_VIEWPORT_MARGIN = 100

class MyGame(arcade.Window):
    """
    Main application class.
    """

    def __init__(self):

        # Call the parent class and set up the window
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
        
        self.coin_list = None
        self.wall_list = None
        self.player_list = None
        self.enemy_list = None
        self.player_sprite = None
        self.allsprites = None
        self.physics_engine = None
        self.view_bottom = 0
        self.view_left = 0
        self.background = None
        self.total_time = 0.0
        # Load sounds
        self.collect_coin_sound = arcade.load_sound("sounds/coin1.wav")
        self.jump_sound = arcade.load_sound("sounds/jump1.wav")
        self.gem_grab_sound = arcade.load_sound("sounds/Minecraft-levelup.mp3")
        self.enemy_touch = arcade.load_sound("sounds/Minecraft-zombie2.mp3")
        self.score = 0
        self.numcoins = 0
        arcade.set_background_color(arcade.csscolor.DARK_ORANGE)

    def setup_lvl_1(self):
        """ Set up the game here. Call this function to restart the game. """
        self.total_time = 0.0
        self.score = 0
        self.view_bottom = 0
        self.view_left = 0
        self.background = arcade.load_texture("2dback.jpg")
        # Create the Sprite lists
        self.player_list = arcade.SpriteList()
        self.wall_list = arcade.SpriteList()
        self.coin_list = arcade.SpriteList()
        self.gem_list = arcade.SpriteList()
        self.enemy_list = arcade.SpriteList()
        self.allsprites = arcade.SpriteList()
 # Set up the player, specifically placing it at these coordinates.
        self.player_sprite = arcade.Sprite("images/player_1/player_stand.png", CHARACTER_SCALING)
        self.player_sprite.center_x = 64
        self.player_sprite.center_y = 108
        self.player_list.append(self.player_sprite)
        self.allsprites.append(self.player_sprite)
         # Create the ground
        # This shows using a loop to place multiple sprites horizontally
        for x in range(0, 2500, 64):
            wall = arcade.Sprite("images/tiles/grassMid.png", TILE_SCALING)
            wall.center_x = x
            wall.center_y = 32
            self.wall_list.append(wall)
            self.allsprites.append(wall)
        # Put some crates on the ground
        # This shows using a coordinate list to place sprites
        coordinate_list = [[534, 96],
                           [278, 96],
                           [788, 96],
                           [534, 162],
                           [790, 162],
                           [790, 226]]

        for coordinate in coordinate_list:
            # Add a crate on the ground
            wall = arcade.Sprite("images/tiles/boxCrate_double.png", TILE_SCALING)
            wall.position = coordinate
            self.wall_list.append(wall)
            self.allsprites.append(wall)
        for x in range(278, 1250, 256):
            coin = arcade.Sprite("images/items/coinGold.png", COIN_SCALING)
            coin.center_x = x
            if coin.center_x == 278:
                coin.center_y = 162
            elif coin.center_x > 530 and coin.center_x < 550:
                coin.center_y = 226
            elif coin.center_x == 790:
                coin.center_y = 280 
            else:
                coin.center_y = 96
            self.coin_list.append(coin)
            self.allsprites.append(coin)
        gemlist = [[440, 96],
                   [680, 96]]
        for cord in gemlist:
            gem = arcade.Sprite("images/items/gemBlue.png", .5)
            gem.position = cord
            self.gem_list.append(gem)
            self.allsprites.append(gem)
        self.enemy_slime = arcade.Sprite("images/enemies/slimeBlock.png", .25)
        self.enemy_slime.center_x = 750
        self.enemy_slime.center_y = 96
        self.enemy_list.append(self.enemy_slime)
        self.allsprites.append(self.enemy_slime)
        #create physics engine
        self.physics_engine = arcade.PhysicsEnginePlatformer(self.player_sprite, self.wall_list, GRAVITY)


        pass

    def on_draw(self):
        """ Render the screen. """
#Clear screen to BG
        arcade.start_render()
        # Code to draw the screen goes here
        # Draw our sprites
        self.wall_list.draw()
        self.coin_list.draw()
        self.player_list.draw()
        self.enemy_list.draw()
        self.gem_list.draw()
        arcade.draw_texture_rectangle(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2,
                                      SCREEN_WIDTH, SCREEN_HEIGHT, self.background)
        #time
         # Calculate minutes
        minutes = int(self.total_time) // 60
        # Calculate seconds by using a modulus (remainder)
        seconds = int(self.total_time) % 60

        # Figure out our output
        output = f"Time: {minutes:2d}:{seconds:2d}"

        # Output the timer text.
        arcade.draw_text(output, 300, 300, arcade.color.BLACK, 30)
        #Draw score
        score_text = f"Score: {self.score}"
        arcade.draw_text(score_text, 10 + self.view_left, 10 + self.view_bottom, arcade.csscolor.WHITE, 18)
        #Coins left
        coin_text = f"Coins left: {self.numcoins}"
        arcade.draw_text(coin_text, 10 + self.view_left, 30 + self.view_bottom, arcade.csscolor.WHITE, 18)
    def on_key_press(self, key, modifiers):
        #Called whenever a key is pressed. I N P U T S
        if key == arcade.key.UP or key == arcade.key.W:
             if self.physics_engine.can_jump():
                self.player_sprite.change_y = PLAYER_JUMP_SPEED
                arcade.play_sound(self.jump_sound)
        elif key == arcade.key.RIGHT or key == arcade.key.D:
            self.player_sprite.change_x = PLAYER_MOVEMENT_SPEED
        elif key == arcade.key.LEFT or key == arcade.key.A:
            self.player_sprite.change_x = -PLAYER_MOVEMENT_SPEED
    #To make it stop and make the movement fluid, we need to define then call a function for key release
    def on_key_release(self, key, modifiers):
        if key == arcade.key.RIGHT or key == arcade.key.D:
            self.player_sprite.change_x = 0
        elif key == arcade.key.LEFT or key == arcade.key.A:
            self.player_sprite.change_x = 0
        
    def update(self, delta_time):
        """Movement and Game Logic"""
        #if self.numcoins == 0:
            #self.game_over == True
            #arcade.draw_text("You win!", 400 + self.view_left, 500 + self.view_bottom, arcade.dcsscolor.CORNFLOWER_BLUE, 30)
        #Calls update on all sprites
        self.physics_engine.update()
        #time
        self.total_time += delta_time
         
         #See if we hit any coins/gems
        coin_hit_list = arcade.check_for_collision_with_list(self.player_sprite,
                                                             self.coin_list)
        gem_hit_list = arcade.check_for_collision_with_list(self.player_sprite,
                                                             self.gem_list)
        enemy_hit_list = arcade.check_for_collision_with_list(self.player_sprite, self.enemy_list)
        # Loop through each coin & gem we hit (if any) and remove it
        for coin in coin_hit_list:
            # Remove the coin
            coin.remove_from_sprite_lists()
            # Play a sound
            arcade.play_sound(self.collect_coin_sound)
            # Add one to the score
            self.score += 1
        for gem in gem_hit_list:
            gem.remove_from_sprite_lists()
            arcade.play_sound(self.gem_grab_sound)
            self.score += 5
        if len(enemy_hit_list) > 0:
            self.game_over == True
            
        self.numcoins = len(self.coin_list)
        # --- Manage Scrolling ---

        # Track if we need to change the viewport

        changed = False

        # Scroll left
        left_boundary = self.view_left + LEFT_VIEWPORT_MARGIN
        if self.player_sprite.left < left_boundary:
            self.view_left -= left_boundary - self.player_sprite.left
            changed = True

        # Scroll right
        right_boundary = self.view_left + SCREEN_WIDTH - RIGHT_VIEWPORT_MARGIN
        if self.player_sprite.right > right_boundary:
            self.view_left += self.player_sprite.right - right_boundary
            changed = True

        # Scroll up
        top_boundary = self.view_bottom + SCREEN_HEIGHT - TOP_VIEWPORT_MARGIN
        if self.player_sprite.top > top_boundary:
            self.view_bottom += self.player_sprite.top - top_boundary
            changed = True

        # Scroll down
        bottom_boundary = self.view_bottom + BOTTOM_VIEWPORT_MARGIN
        if self.player_sprite.bottom < bottom_boundary:
            self.view_bottom -= bottom_boundary - self.player_sprite.bottom
            changed = True

        if changed:
            # Only scroll to integers. Otherwise we end up with pixels that
            # don't line up on the screen
            self.view_bottom = int(self.view_bottom)
            self.view_left = int(self.view_left)

            # Do the scrolling
            arcade.set_viewport(self.view_left,
                                SCREEN_WIDTH + self.view_left,
                                self.view_bottom,
                                SCREEN_HEIGHT + self.view_bottom)

def main():
    """ Main method """
    window = MyGame()
    window.setup_lvl_1()
    arcade.run()


# if __name__ == "__main__":
#     main()
    

In [ ]:
main()

In [1]:
%pwd

'C:\\Users\\gania\\Desktop\\GeoDash Python'

In [1]:
"""
Platformer Game
"""
import pyglet
import playsound
import arcade
import os
import time
# Constants (alter for your game or delete)
SCREEN_WIDTH = 2048
SCREEN_HEIGHT = 1536
SCREEN_TITLE = "Geometry Dash bootleg"
CHARACTER_SCALING = 1
TILE_SCALING = 1
COIN_SCALING = 1
SPRITE_PIXEL_SIZE = 128
GRID_PIXEL_SIZE = (SPRITE_PIXEL_SIZE * TILE_SCALING)
PLAYER_MOVEMENT_SPEED = 17
GRAVITY = 2
PLAYER_JUMP_SPEED = 24
PLAYER_START_X = 64
PLAYER_START_Y = 961

GAME_RUNNING = 2
GAME_OVER = 3
# How many pixels to keep as a minimum margin between the character
# and the edge of the screen.
LEFT_VIEWPORT_MARGIN = 128
RIGHT_VIEWPORT_MARGIN = 1700
BOTTOM_VIEWPORT_MARGIN = 950
TOP_VIEWPORT_MARGIN = 300

class MyGame(arcade.Window):
    """
    Main application class.
    """

    def __init__(self):

        # Call the parent class and set up the window
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
        
        self.wall_list = None
        self.player_list = None
        self.spike_or_bad_list = None
        self.player_sprite = None
        self.allsprites = None
        self.physics_engine = None
        self.view_bottom = 0
        self.view_left = 0
        self.danube = arcade.load_sound("1geosong.wav")
        self.end_of_map = 0
        self.current_state = GAME_RUNNING
        arcade.set_background_color(arcade.csscolor.CORNFLOWER_BLUE)

    def setup_lvl_1(self):
        """ Set up the game here. Call this function to restart the game. """
        self.view_bottom = 0
        self.view_left = 0
        # Create the Sprite lists
        self.player_list = arcade.SpriteList()
        self.wall_list = arcade.SpriteList()
        self.spike_or_bad_list = arcade.SpriteList()
        self.allsprites = arcade.SpriteList()
        self.spike = arcade.Sprite("spike1.png")
        self.spike_or_bad_list.append(self.spike)
        self.allsprites.append(self.spike)
        """"""
        arcade.play_sound(self.danube)
        """"""
 # Set up the player, specifically placing it at these coordinates.
        self.player_sprite = arcade.Sprite("geodash-player.png", CHARACTER_SCALING)
        self.player_sprite.center_x = 64
        self.player_sprite.center_y = 961
        self.player_list.append(self.player_sprite)
        self.allsprites.append(self.player_sprite)
        # --- Load in a map from the tiled editor ---
        platforms_layer_name = 'Platforms'
        bad_layer_name = 'Spikes'
        # Name of map file to load
        map_name = f"GeoDash.tmx"
        # Read in the tiled map
        my_map = arcade.tilemap.read_tmx(map_name)
        # Calculate the right edge of the my_map in pixels
        self.end_of_map = my_map.map_size.width * GRID_PIXEL_SIZE
        # -- Platforms
        self.wall_list = arcade.tilemap.process_layer(my_map, platforms_layer_name, TILE_SCALING)

        # -- Coins
        #self.coin_list = arcade.tilemap.process_layer(my_map, coins_layer_name, TILE_SCALING)
        
        # -- Spikes or Bad Layer
        self.spike_or_bad_list = arcade.tilemap.process_layer(my_map, bad_layer_name, TILE_SCALING)
        

        #create physics engine
        self.physics_engine = arcade.PhysicsEnginePlatformer(self.player_sprite, self.wall_list, GRAVITY)


        pass
    def on_draw(self):
        """ Render the screen. """
        #Clear screen to BG
        arcade.start_render()
        self.wall_list.draw()
        self.player_list.draw()
        self.spike_or_bad_list.draw()
        self.player_sprite.change_x = PLAYER_MOVEMENT_SPEED
        
    def on_key_press(self, key, modifiers):
        #Called whenever a key is pressed. I N P U T S
        if key == arcade.key.UP or key == arcade.key.W and self.current_state == GAME_RUNNING:
             if self.physics_engine.can_jump():
                self.player_sprite.change_y = PLAYER_JUMP_SPEED
                #arcade.play_sound(self.jump_sound)
        #if key == arcade.key.S:
            #import sys
            #subprocess.call(sys.executable + ' "' + os.path.realpath(__file__) + '"')
            #main()
            

    #def on_key_release(self, key, modifiers):
        
        
    def update(self, delta_time):
        """Movement and Game Logic"""
        if self.current_state == GAME_RUNNING:
            
            #Calls update on all sprites
            
            self.physics_engine.update()
            self.player_list.update()
            # --- Manage Scrolling ---

            # Track if we need to change the viewport

            changed = False
            
            # Did the player fall off the map?
            
            if self.player_sprite.center_y < -500:
                """"""
                #arcade.stop_sound
                """"""
                #arcade.play_sound("1geosong.wav")
                """"""
                self.player_sprite.center_x = PLAYER_START_X
                self.player_sprite.center_y = PLAYER_START_Y
                
                # Set the camera to the start
                
                self.view_left = 0
                self.view_bottom = 0
                changed = True
            if self.player_sprite.center_x >= self.end_of_map - 3072:
                self.player_sprite.change_x = 0
                self.player_sprite.change_y = 0
                arcade.close_window()
                self.current_state = GAME_OVER
            # Did the player touch something they should not?
            
            if arcade.check_for_collision_with_list(self.player_sprite, self.spike_or_bad_list):
                #PUT EXPLOSION HERE
            
                """"""
                #arcade.stop_sound
                """"""
                #arcade.play_sound("1geosong.wav")
                """"""
                self.player_sprite.change_x = 0
                self.player_sprite.change_y = 0
                self.player_sprite.center_x = PLAYER_START_X
                self.player_sprite.center_y = PLAYER_START_Y
            
                # Set the camera to the start
                
                self.view_left = 0
                self.view_bottom = 0
                changed = True
            
        # Scroll left
        left_boundary = self.view_left + LEFT_VIEWPORT_MARGIN
        if self.player_sprite.left < left_boundary:
            self.view_left -= left_boundary - self.player_sprite.left
            changed = True

        # Scroll right
        right_boundary = self.view_left + SCREEN_WIDTH - RIGHT_VIEWPORT_MARGIN
        if self.player_sprite.right > right_boundary:
            self.view_left += self.player_sprite.right - right_boundary
            changed = True

        # Scroll up
        top_boundary = self.view_bottom + SCREEN_HEIGHT - TOP_VIEWPORT_MARGIN
        if self.player_sprite.top > top_boundary:
            self.view_bottom += self.player_sprite.top - top_boundary
            changed = True

        # Scroll down
        bottom_boundary = self.view_bottom + BOTTOM_VIEWPORT_MARGIN
        if self.player_sprite.bottom < bottom_boundary:
            self.view_bottom -= bottom_boundary - self.player_sprite.bottom
            changed = True

        if changed:
            # Only scroll to integers. Otherwise we end up with pixels that
            # don't line up on the screen
            self.view_bottom = int(self.view_bottom)
            self.view_left = int(self.view_left)

            # Do the scrolling
            arcade.set_viewport(self.view_left,SCREEN_WIDTH + self.view_left, self.view_bottom, SCREEN_HEIGHT + self.view_bottom)
def main():
    """ Main method """
    window = MyGame()
    window.setup_lvl_1()
    arcade.run()



# if __name__ == "__main__":
     #main()

In [2]:
main()

AttributeError: 'NoneType' object has no attribute 'width'

In [2]:
main()

AttributeError: 'Sound' object has no attribute 'pause'

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\gania\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\gania\Anaconda3\lib\asyncio\selector_events.py", line 114, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\gania\Anaconda3\lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\gania\Anaconda3\lib\asyncio\selector_events.py", line 114, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed b

In [3]:
main()

AttributeError: 'Sound' object has no attribute 'pause'